# <font color='blue'>Preface</font>
This <a href='http://localhost/jforum/posts/list/4104.page'>Chapter</a> covers 
* The null reference, or “the billion-dollar mistake” 
* Alternatives to null references 
* Developing an Option data type for optional data 
* Applying functions to optional values 
* Composing optional values 
* Option use cases

<b>Representing optional data in computer programs has always been a problem</b>. The concept of optional data is very simple in everyday life. Representing the absence of something when this something is contained in a container is easy—whatever it is, it can be represented by an empty container. An absence of apples can be represented by an empty apple basket. The absence of gasoline in a car can be visualized as an empty gas tank. Representing the absence of data in computer programs is more difficult. Most data is represented as a reference pointing to it, so the most obvious way to represent the absence of data is to use a pointer to nothing. This is what a null pointer is. 

In Python, a variable is a pointer to a value. Variables may be created <b><a href='https://docs.python.org/2/library/constants.html#None'>None</a></b> , and they may then be changed to point to values. They can even be changed again to point to null if data is removed. To handle optional data, in this chapter, you’ll develop the class called <b><font color="blue">Option</font></b>. The goal of this chapter is to learn how this kind of structure works in handling optional data.

# <font color='blue'>Problems with the null pointer</font>
One of the most frequent bugs in imperative programs is the <font color='blue'><b>AttributeError</b></font>. This error is raised when an identifier is dereferenced and found to be pointing to nothing. In other words, some data is expected but is found missing. Such an identifier is said to be pointing to null. The null reference was invented in 1965 by Tony Hoare while he was designing the ALGOL object-oriented language. Here’s what he said 44 years later:


# <font color='blue'>The Option data type </font>
The <b><font color='blue'>Option</font></b> data type you’ll create in this chapter will be very similar to the List data type. Using an <b><font color='blue'>Option</font></b> type for optional data allows you to compose functions even when the data is absent (<font color='brown'>see figure 6.1</font>). It will be implemented as an abstract class, <b><font color='blue'>Option</font></b>, containing two private subclasses representing the presence and the absence of data. The subclass representing the absence of data will be called <b><font color='blue'>N</font></b> (<font color='brown'>Stands for None</font>), and the subclass representing the presence of data will be called Some. A Some will contain the corresponding data value.
<img src='https://3.bp.blogspot.com/-55VlPQ6TLEk/WkuDRgPMWII/AAAAAAAAXII/Yj_WnnSeS4UvBMzDflzhwJoz4E8ecuZoQCLcBGAs/s1600/4104_f6-1.PNG'/>
<b>Figure 6.1. Without the Option type, composing functions wouldn’t produce a function because the resulting program would potentially throw an Exception.</b>


## <font color='green'>Getting a value from an Option</font>
The first thing you’ll need is a way to retrieve the value in an <b><font color='blue'>Option</font></b>. One frequent use case when data is missing is to use a default value. The <font color='blue'>getOrElse</font> method that will return either the contained value if it exists:

In [2]:
from fp import *

print("{}".format(sys.version_info))
print("Retrieve default: {}".format(Option.none().getOr('default value')))
print("Retrieve value: {}".format(Option.some('some value').getOr('default value')))

def getDefault():
    print('api:getDefault is called')
    return 0

# What if we only want api:getDefault to be called only when needed?
print("Retrieve: {}".format(Option.some(123).getOr(getDefault())))
print("Retrieve: {}".format(Option.none().getOr(getDefault())))

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)
Retrieve default: default value
Retrieve value: some value
api:getDefault is called
Retrieve: 123
api:getDefault is called
Retrieve: 0


Or you can use lazy evaluation for the <font color='blue'>getOrElse</font> method parameter:

In [2]:
print("Retrieve: {}".format(Option.some(123).getOrElse(getDefault)))
print("Retrieve: {}".format(Option.none().getOrElse(getDefault)))

Retrieve: 123
api:getDefault is called
Retrieve: 0


## <font color='green'>Applying functions to optional values</font>
One very important method in <b><font color='blue'>List</font></b> is the <font color='blue'>map</font> method, which allows you to apply a function from A to B to each element of a list of A, producing a list of B. Considering that an <b><font color='blue'>Option</font></b> is like a list containing at most one element, you can apply the same principle. 

In [3]:
some = Option.some(1)
none = Option.none()
double = lambda e: e * 2
print("Double {}={}".format(some, some.map(double)))
print("Double {}={}".format(none, none.map(double)))

Double Some(1)=Some(2)
Double None()=None()


## <font color='green'>Dealing with Option composition</font>
As you’ll soon realize, functions from A to B aren’t the most common ones in functional programming. 
At first you may have trouble getting acquainted with functions returning optional values. 
After all, it seems to involve extra work to wrap values in <b><font color='blue'>Some</font></b> instances and later retrieve these values. 
But with further practice, you’ll see that these operations occur only rarely. 
<b>When chaining functions to build a complex computation, you’ll often start with a value that’s returned by some previous computation and pass the result to a new function without seeing the intermediate result. In other words, you’ll more often use functions from A to <font color='blue'>Option&lt;B></font> than functions from A to B.</b> 

<font color='blue'>flatMap</font> instance method that takes as an argument a function from A to <b><font color='blue'>Option&lt;B></font></b> and returns an <b><font color='blue'>Option&lt;B></font></b>:

In [4]:
some = Option.some(1)
none = Option.none()
addOne = lambda e: Option.some(e + 1)
double = lambda e: Option.some(e * 2)

print("some.flatMap(addOne).flatMap(double)={}".format(some.flatMap(addOne).flatMap(double)))
print("none.flatMap(addOne).flatMap(double)={}".format(none.flatMap(addOne).flatMap(double)))

some.flatMap(addOne).flatMap(double)=Some(4)
none.flatMap(addOne).flatMap(double)=None()


Most of the case, there’s no need to “get” the value. Instead, you need to get the default value wrapped in <b><font color='blue'>Option</font></b> for chaining functions. Then <font color='blue'>orElse</font> will meet your need. For example:

In [5]:
some = Option.some(1)
none = Option.none()
def df():
    print('api:df is called')
    return Option.some(-1)

print("some.OrElse(df)={}".format(some.orElse(df)))
print("none.OrElse(df)={}".format(none.orElse(df)))

some.OrElse(df)=Some(1)
api:df is called
none.OrElse(df)=Some(-1)


More useful, you can use <font color='blue'>filter</font> to pass in the filter function which will return the original <b><font color='blue'>Some</font></b> object if True; otherwise, <b><font color='blue'>N</font></b> is returned:

In [6]:
some1 = Option.some(1)
some2 = Option.some(2)
none = Option.none()
bt1 = lambda e: e > 1  # Bigger than one

print("some1.filter(bt1)={}".format(some1.filter(bt1)))
print("some2.filter(bt1)={}".format(some2.filter(bt1)))
print("none.filter(bt1)={}".format(none.filter(bt1)))

some1.filter(bt1)=None()
some2.filter(bt1)=Some(2)
none.filter(bt1)=None()


## <font color='green'>Option use cases</font>
The best way to use <b><font color='blue'>Option</font></b> is through composition. To do this, you must create all the necessary methods for all use cases. These use cases correspond to what you’d do with the value after testing that it’s not null. You could do one of the following: 
* Use the value as the input to another function 
* Apply an effect to the value 
* Use the value if it’s not null, or use a default value to apply a function or an effect

Below is a classis example in using <b><font color='blue'>Option</font></b>:

In [3]:
class TMap:
    def __init__(self):
        self.map = {}
        
    def get(self, k):
        if k in self.map:
            return Option.some(self.map[k])
        else:
            return Option.none()
        
    def put(self, k, v):
        self.map[k] = v
        return self
        
    def removeKey(self, k):
        if k in self.map:
            ov = self.map[k]
            del self.map[k]
            return Option.some(ov)
        else:
            return Option.none()
        
class Toon:
    def __init__(self, fn, sn, email=Option.none()):
        self.fn = fn
        self.sn = sn
        if isinstance(email, Option):
            self.email = email
        else:
            self.email = Option.some(email)
        
    def __str__(self):
        return "{}/{} ({})".format(self.fn, self.sn, email.getOrEalse(lambda: "No data"))

toons = TMap()
for k, t in [("Mickey", Toon("Mickey", "Mouse", "mickey@disney.com")),
             ("Minnie", Toon("Minnie", "Mouse")),
             ("Donald", Toon("Donald", "Duck", "donald@disney.com"))]:    
    toons.put(k, t)
    
def getEmail(e):
    return e.email

def dft():
    return "No data"

mickey = toons.get("Mickey").flatMap(getEmail)
minnie = toons.get("Minnie").flatMap(getEmail)
goofy  = toons.get("Goofy").flatMap(getEmail)

print("Mickey: {}".format(mickey.getOrElse(dft)))
print("Minnie: {}".format(minnie.getOrElse(dft)))
print("Goofy: {}".format(goofy.getOrElse(dft)))

Mickey: mickey@disney.com
Minnie: No data
Goofy: No data


# <font color='blue'>Miscellaneous utilities for Option</font>
In order to make <font color='blue'><b>Option</b></font> as useful as possible, you need to add some utility methods. Some of these methods are a must, and others are questionable because their use is not in the spirit of functional programming. You nevertheless must consider adding them. You may need a method to test whether an Option is a None or a Some. You may also need an equals method for comparing options, in which case you mustn’t forget to define a compatible hashCode method.

## <font color='green'>Testing for Some or None</font>
Until now, you haven’t needed to test an option to know whether it was a <font color=blue><b>Some</b></font> or a <font color=blue><b>None</b></font>. Ideally, you should never have to do this. In practice, though, there are times when it’s simpler to use this trick than to resort to real functional techniques. For example, you defined the <i>map2</i> method as:


In [5]:
def map2(opa, opb, f):
    return opa.flatMap(lambda ax: opb.map(lambda bx: f(ax, bx)))

opa = Option.some(1)
opb = Option.some(2)
map2(opa, opb, lambda a, b: a + b).getOrElse(lambda e: 0)

3

This is very smart, and because you want to look smart, you might prefer this solution. But some may find the following version simpler to understand: 

In [6]:
def map2_v2(opa, opb, f):
    return Option.some(f(opa.getOrThrow(), opb.getOrThrow())) if opa.isSome and opb.isSome else opOption.none()

map2_v2(Option.some(1), Option.some(2), lambda a, b: a + b).getOrThrow()

3

If you want to test this code, you’ll have to define the <font color='blue'>isSome</font> method first, <b>but this is not to encourage you to use this nonfunctional technique</b>. You should always prefer the first form, but you should also understand fully the relation between the two forms. Besides, you’ll probably find yourself needing the <font color='blue'>isSome</font> method someday. 

## <font color='green'>equals and hashcode </font>
Much more important are the definitions of the <a href='https://docs.python.org/2/reference/datamodel.html#object.__eq__'>\_\_eq\_\_</a> and <a href='https://docs.python.org/2/reference/datamodel.html#object.__hash__'>\_\_hash\_\_</a> methods. As you know, these methods are strongly related and must be consistently defined. If equals is true for two instances of <font color='blue'><b>Option</b></font>, their hashcode methods should return the same value. (The inverse is not true. Objects having the same hashcode may not always be equal.) 

# <font color='blue'>How and when to use Option</font> 
Here’s how Brian Goetz, Java language architect at Oracle, answered a question about this subject on Stack Overflow. The question was “Should Java 8 getters return optional types?” Here is Brian Goetz’s answer: 
The full discussion may be read at http://mng.bz/Rkk1. 
```python
Of course, people will do what they want. But we did have a clear intention when adding this feature, and it was not to be a general purpose Maybe or Some type, as much as many people would have liked us to do so. Our intention was to provide a limited mechanism for library method return types where there needed to be a clear way to represent “no result” and using null for such was overwhelmingly likely to cause errors. 

For example, you probably should never use it for something that returns an array of results, or a list of results; instead return an empty array or list. You should almost never use it as a field of something or a method parameter. I think routinely using it as a return value for getters would definitely be over-use. There’s nothing wrong with Optional that it should be avoided, it’s just not what many people wish it were, and accordingly we were fairly concerned about the risk of zealous over-use. 
````
This is a very important answer that deserves some reflection. First of all, and this might be the most important part, “people will do what they want.” Nothing to add here. Just do what you want. This doesn’t mean you should do whatever you want without thinking. But feel free to try every solution that comes to mind. You shouldn’t refrain from using Optional in a particular way just because it wasn’t intended to be used that way. Imagine the first man who ever thought about grabbing a stone to hit something with more strength. He had two options (pun intended!): refraining from doing it because stones had obviously not been intended to be used as hammers, or just trying it. 

## <font color='green'>When to use getOrThrow </font>
<b>The correct advice is to avoid getOrThrow as much as possible. As a rule of thumb, each time you find yourself using this method outside of the <font color='blue'>Option</font> class, you should consider whether there’s another way to go</b>. Using <font color='blue'>getOrThrow</font> is exiting the functional safety of the <font color='blue'><b>Option</b></font> class. 

The most important point is the original question: should getters return <font color='blue'><b>Option</b></font>? Generally, they shouldn’t, because properties should be final and initialized at declaration or in constructors, so there’s absolutely no need for getters to return <font color='blue'><b>Option</b></font>. (<font color='brown'>I must admit, however, that initializing fields in constructors doesn’t guarantee that access to properties is impossible before they’re initialized. This is a marginal problem that’s easily solved by making classes final, if possible.</font>) 

<b>But some properties might be optional.</b> For example, a person will always have a first name and a last name, but they might have no email. How can you represent this? By storing the property as an <font color='blue'><b>Option</b></font>. In such cases, the getter will have to return an <font color='blue'><b>Option</b></font>. Saying that “routinely using it as a return value for getters would definitely be over-use” is like saying that a property without a value should be set to null, and the corresponding getter should return null. This completely destroys the benefit of having <font color='blue'><b>Option</b></font>. 

What about methods that take <font color='blue'><b>Option</b></font> as their argument? In general, this should not occur. To compose methods returning <font color='blue'><b>Option</b></font>, you shouldn’t use methods that take <font color='blue'><b>Option</b></font> as their argument. For example, to compose the two following methods, you don’t need to change the methods to make them accept <font color='blue'><b>Option</b></font> as their argument: 
```python
def validate(String name):
    if name.startswith('J'):
        return Option.some(name)
    else:
        return Option.none()
  
def toUpper(name):
    return Option.some(name.upper())
```

The functional way to compose these methods is as follows:
```python
Option.some('Ken').flatMap(validate).flatMap(toUpper).getOrElse(lambda: 'No name')  # print 'No name'
Option.some('John').flatMap(validate).flatMap(toUpper).getOrElse(lambda: 'No name')  # print 'John'
```